In [ ]:
import pandas as pd
from rdkit import Chem

In [19]:
df = pd.read_csv('C:/Users/Enrique/Desktop/raw_data_HLM.txt', sep='\t')

#@title Sanitization in the database

#Create a list to include the sanitized SMILES
new_smiles = []
#Iterate on the list of SMILES
for smi in df['SMILES']:
    try:
        #Sanitize if possible
        sanitized_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi))
    except:
        #Use None if not (None without "" because it is an empty value not a text)
        sanitized_smi = None

    #Append the value to the new_smiles list
    new_smiles.append(sanitized_smi)

#Include the new_smiles list as a new 'san_SMILES' column
df['san_SMILES'] = new_smiles

df = df[df['san_SMILES'].notna()] #You can use the notna() method inside the brackets

df

,SMILES,y,san_SMILES
0,C(Oc1ccccc1)[C@@H]2CC[C@H]3CN(CCN3C2)c4ncccn4,1,c1ccc(OC[C@@H]2CC[C@H]3CN(c4ncccn4)CCN3C2)cc1
1,Fc1ccc(OC[C@@H]2CC[C@H]3CN(CCN3C2)c4ncccn4)cc1,1,Fc1ccc(OC[C@@H]2CC[C@H]3CN(c4ncccn4)CCN3C2)cc1
2,Fc1ccc(OC[C@@H]2CC[C@H]3CN(CCN3C2)c4ncc(F)cn4)cc1,1,Fc1ccc(OC[C@@H]2CC[C@H]3CN(c4ncc(F)cn4)CCN3C2)cc1
3,Cl.CCCCCCSc1nccnc1O[C@H]2CN3CCC2C3,0,CCCCCCSc1nccnc1O[C@H]1CN2CCC1C2.Cl
4,Cc1ccc(N2CCN(C[C@@H]3C[C@H]3c4ccccc4)CC2)c(C)c1,1,Cc1ccc(N2CCN(C[C@@H]3C[C@H]3c3ccccc3)CC2)c(C)c1
...,...,...,...
3649,O\N=C(/NC1CCCCC1)\c2ccccc2c3ccccc3,0,O/N=C(\NC1CCCCC1)c1ccccc1-c1ccccc1
3650,O\N=C(/NCC1CC1)\c2ccccc2c3ccccc3,0,O/N=C(\NCC1CC1)c1ccccc1-c1ccccc1
3651,CCCCCC(C)NCc1coc(n1)c2ccc(cc2)c3ccccc3,0,CCCCCC(C)NCc1coc(-c2ccc(-c3ccccc3)cc2)n1
3652,N=C(NC1CCCCC1)c2ccccc2c3ccccc3,0,N=C(NC1CCCCC1)c1ccccc1-c1ccccc1


In [20]:
#@title Apply salt remover to the dataset
from rdkit.Chem.SaltRemover import SaltRemover

smi_no_salts = []
for smi in df['SMILES']:
    mol = Chem.MolFromSmiles(smi)
    remover = SaltRemover(defnData='[Na,Cl,K,O,OH,Fe,F,H,Al,Mg,Co,Ti,NH4,Mn,Si,Ca,Au,I,Hg,Mo,Zn,Br,Ag,Sr,Cu,Bi,S,Li,NH3,He,Y,Ar,Ba,La]')
    res = remover.StripMol(mol)
    new_smi = Chem.MolToSmiles(res)
    smi_no_salts.append(new_smi)

df['NO SALTS'] = smi_no_salts

df = df[df['NO SALTS']!= '']

#@title Inorganic removal in the dataset
allowed_atoms = ['H', 'N', 'C', 'O', 'P', 'S', 'Si', 'F', 'Cl', 'Br', 'I']

non_allowed_indx = []
#We reset the index to be sure that the index corresponds to the actual position of the SMILES in the list
df.reset_index(drop=True, inplace=True)

#Use enumerate to obtain the index in additon of the SMILES
for i, smi in enumerate(df['NO SALTS']):

    mol = Chem.MolFromSmiles(smi)
    for atom in mol.GetAtoms():
        atom_symbol = atom.GetSymbol()
        if atom_symbol not in allowed_atoms:
           non_allowed_indx.append(i)  # Append the index
           break

df.drop(non_allowed_indx,inplace=True)
df.reset_index(drop=True, inplace=True)

df

#@title Mixtures removal in the dataset
mixtures_indx = []

for i, smi in enumerate(df['NO SALTS']):
    if '.' in smi:
        mixtures_indx.append(i)

df.drop(mixtures_indx, inplace=True)
df.reset_index(drop=True, inplace=True)

#@title Removal of duplicates in the dataset
df.drop_duplicates(subset='NO SALTS', inplace=True, keep=False)
df

df_final = df[['NO SALTS', 'y']]
df_final.columns = ['SMILES', 'y']

In [22]:
df_final.to_csv('C:/Users/Enrique/Desktop/raw_data.csv', sep=';', index=None)

In [26]:
def compare_compounds(df1, df2):
    """
    Compara los compuestos en dos DataFrames de pandas, e imprime los compuestos que son similares entre ambos,
    verificando además si el valor de la variable 'y' es el mismo para estos compuestos similares.

    Parámetros:
    - df1: Primer DataFrame de pandas que contiene columnas con compuestos (SMILES) y sus valores de 'y'.
    - df2: Segundo DataFrame de pandas que contiene columnas con compuestos (SMILES) y sus valores de 'y'.
    """
    compuestos_df1 = df1.set_index('SMILES')['y'].to_dict()
    compuestos_df2 = df2.set_index('SMILES')['y'].to_dict()

    # Encontrar la intersección de ambos conjuntos para identificar compuestos similares
    compuestos_similares = set(compuestos_df1.keys()).intersection(compuestos_df2.keys())
    
    if compuestos_similares:
        print("Compuestos similares encontrados:", len(compuestos_similares))
        for compuesto in compuestos_similares:
            valor_y_df1 = compuestos_df1[compuesto]
            valor_y_df2 = compuestos_df2[compuesto]
            # Comparar los valores de 'y' para los compuestos similares
            if valor_y_df1 == valor_y_df2:
                #print(f"Compuesto: {compuesto}, Valor 'y' coincidente: {valor_y_df1}")
                None
            else:
                print(f"Compuesto: {compuesto}, Valor 'y' en df1: {valor_y_df1}, Valor 'y' en df2: {valor_y_df2}")
    else:
        print("No se encontraron compuestos similares.")

# Ejemplo de uso
# Supongamos que tienes un DataFrame 'df' y un archivo 'compuestos.txt'
df = df_final
df_cured = pd.read_csv('C:/Users/Enrique/Documents/ProtoADME para Enrique/ProtoADME_METABOLISM/human_liver_microsomal/v4/human_liver_microsomal.csv', sep=';')
compare_compounds(df, df_cured)

Compuestos similares encontrados: 2996
Compuesto: O=C1CN(S(=O)(=O)c2ccc(Cl)c(Cl)c2)CCN1c1ncccc1C(F)(F)F, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: CC(C)CCn1nc(-c2cc[nH]c2)c(O)c(C2=NS(=O)(=O)c3cc(NS(C)(=O)=O)ccc3N2)c1=O, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: COc1ccc(CCN2C(=O)N(NS(C)(=O)=O)C[C@H]2c2ccc(OC)cc2)cc1, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: Cc1cc(S(=O)(=O)Nc2cccc(CO)c2)ccc1-c1ccc(F)cc1F, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: CNC[C@H](CC1CCCCC1)NC(=O)N1CCC[C@@H]([C@@H](OCCNC(=O)OC)c2cccc(Cl)c2)C1, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: CC(C)CC(=O)N1CCC(n2cnc3cnc4[nH]ccc4c32)CC1, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: C[C@@H]1CN(c2ccc(F)cc2C(F)(F)F)CCN1S(=O)(=O)c1ccc([C@](O)(C(N)=O)C(F)(F)F)s1, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: CCS(=O)(=O)CNc1cc2cc(c1)Nc1nccc(n1)-c1cccc(c1)OCC/C=C/CN(C)C2, Valor 'y' en df1: 1, Valor 'y' en df2: 0
Compuesto: Cc1nc2c(C(F)(F)F)cccc2n1-c1cccc(Oc2cccc(S(C)